In [ ]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install sklearn

# IMPORTO LE LIBRERIE PRINCIPALI

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import plot_confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier




# MI LEGGO I VARI FILE CHE SONO DI SUPPORTO

In [ ]:
train=pd.read_csv('train.csv')

In [ ]:
test=pd.read_csv('test.csv')

In [ ]:
pokemon=pd.read_csv('pokemon.csv')

# DO UNA RAPIDA OCCHIATA AL DATASET

In [ ]:
pokemon

In [ ]:
train.head()


In [ ]:
test.head()

# CONVERTO I VALORI DI WINNER:
- SE VINCE IL PRIMO VALE 1 
- SE VINCE IL SECONDO VALE 0

In [ ]:
def conv_bin_winner(winner, first_pokemon, second_pokemon):
    if (first_pokemon == winner):
        return 1
    elif (second_pokemon == winner):
        return 0


In [ ]:
train['Winner']= train.apply(lambda x: conv_bin_winner(x['Winner'], x['First_pokemon'], x['Second_pokemon']), axis=1)
test['Winner']= test.apply(lambda x: conv_bin_winner(x['Winner'], x['First_pokemon'], x['Second_pokemon']), axis=1)

# INSERISCO TRAIN E TEST IN UN UNICO DATASET 
IN CUI HO UNA COLONNA CHIAMATA TRAIN IL CUI VALORE VALE 1 SE IL RECORD E' PRESENTE NEL TRAIN, 0 SE IL RECORD E' PRESENTE NEL TEST

In [ ]:
train["train"]=1
test["train"]=0
dataset=pd.concat([train, test], ignore_index=True)

In [ ]:
dataset['train']

In [ ]:
list(dataset)

In [ ]:
pokemon.info()

# ESEGUO UN MERGE FRA IL DATASET "POKEMON" E IL DATASET "DATASET"

In [ ]:
joined_data= dataset.join(pokemon.set_index('#'), on ='First_pokemon', rsuffix="_first")

In [ ]:
joined_data= joined_data.join(pokemon.set_index('#'), on ='Second_pokemon', rsuffix="_second")

# VEDO TUTTE LE FEATURES


In [ ]:
list(joined_data)

### MI STAMPO LA QUANTITA' DEI VALORI CHE ASSUMONO LE VARIE FEATURES

In [ ]:
for c in joined_data.columns:
    print ("Features: ", c)
    print(joined_data[c].value_counts())
    print("\n*****************\n")

# VERIFICO SE CI SONO DEI VALORI MANCANTI ALL'INTERNO DEL DATASET

In [ ]:
joined_data.info()

# VALORI MANCANTI
- Name
- Name_second
- Type 2 
- Type 2_second

In [ ]:
print(joined_data.isnull().sum())

# Type2 ha molti valori mancanti!
quasi la metà considerando che i recordo sono 40000, potrei pensare di dropparla, ma dato che il tipo è una feature che dovrebbe essere rilevante nei Pokemon ai fini della sfida, portei pensare di mettere al posto del valore mancante di Type2 il valore di Type1 

In [ ]:
joined_data['Type 2'].fillna(joined_data['Type 1'],inplace=True)
joined_data['Type 2_second'].fillna(joined_data['Type 1_second'],inplace=True)

# "NAME"  DATO CHE E' UNICA SI PUO' DROPPARE

In [ ]:
print(joined_data.isnull().sum())

In [ ]:
len(pokemon['Name'].unique())

In [ ]:
toDrop = ['Name', 'Name_second']
joined_data = joined_data.drop(toDrop, axis=1)

### MOSTRO IL DATASET IN CUI HO DROPPATO LA FEATURE 'NAME' E HO INSERITO DENTRO AL TYPE2 IL VALORE DEL TYPE1 QUANDO IL TYPE2 MANCAVA

In [ ]:
joined_data.head()

##  Encoding: i modelli ragionano in termini di NUMERI e non di stringhe. 
## Per questo motivo, bisogna convertire i campi object/bool in numeri.

In [ ]:
# modifico alcuni nomi delle feature per comodità

In [ ]:
joined_data.rename(columns={'Type 2':'Type2'},inplace=True)
joined_data.rename(columns={'Type 1':'Type1'},inplace=True)

In [ ]:
joined_data.rename(columns={'Type 1_second':'Type1_second'},inplace=True)
joined_data.rename(columns={'Type 2_second':'Type2_second'},inplace=True)
joined_data.rename(columns={'Sp. Atk':'Sp_Atk'},inplace=True)
joined_data.rename(columns={'Sp. Def':'Sp_Def'},inplace=True)
joined_data.rename(columns={'Sp. Atk_second':'Sp_Atk_second'},inplace=True)
joined_data.rename(columns={'Sp. Def_second':'Sp_Def_second'},inplace=True)

In [ ]:
joined_data.dtypes

## i tipi da convertire sono:
- Type1
- Type2
- Type1_second
- Type2_second
- Legendary
- Legendary_second


In [ ]:
from sklearn.preprocessing import LabelEncoder
Feature = ["Type1", "Type2", "Type1_second","Type2_second", "Legendary","Legendary_second"]
labelEncoder_X = LabelEncoder()
for element in Feature:
    joined_data[element] = labelEncoder_X.fit_transform(joined_data[element])
    
joined_data.head()

# RIDIVIDO IL DATASET TOTALE IN TRAIN E TEST SULLA BASE DELLA DIVISIONE EFFETTUATA NEL PREPARATION

In [ ]:
train= joined_data[joined_data.train == 1].drop("train", axis=1)
test= joined_data[joined_data.train == 0].drop("train", axis=1)

# Mi stampo una heatmap che mi guarda quale sia la correlazione che ho fra le feature:
- valore negativo -> relazione di proporzionalità inversa
- valore positivo -> relazione di proporzionalità diretta
- valore pari a zero-> no correlazione (feature può essere eliminata)

In [ ]:
plt.figure(figsize=(18,10))
correlation = train.corr()
sns.heatmap(correlation, annot = True, linewidth = 2)

Dalla heatmap vediamo che le seguenti feature non sono utili per il nostro scopo:
- First_Pokemon
- Second_Pokemon
- Type1 
- Type2
- Defense 
- Generation
- Type1_second 
- Type2_second
- Defense_second
- Generation_second

Droppo quindi:
- First_Pokemon 
- Second_Pokemon
- Type1 
- Type2
- Type1_second 
- Type2_second

In [ ]:
toDrop = ['First_pokemon','Second_pokemon','Type1', 'Type2','Type1_second', 'Type2_second']
train = train.drop(toDrop, axis=1)

In [ ]:
toDrop = ['First_pokemon','Second_pokemon','Type1', 'Type2','Type1_second', 'Type2_second' ]
test = test.drop(toDrop, axis=1)

## Mi stampo un grafico che mostra quanti e come sono distribuiti i valori per Winner nel dataset 

# WINNER=1 --> vince il primo
# WINNER=0 --> vince il secondo

In [ ]:
sns.countplot(x=train['Winner'])

In [ ]:
sns.countplot(x=test['Winner'])

#### Notiamo che i valori sono distribuiti in maniera abbastanza omogenea sia nel dataset di train sia nel dataset di test sembrerebbe inoltre che chi attacca per secondo tende a vincere leggermente di più

# Vedo se si può assumere da questi grafici l'importanza di alcune feature del dataset

### - Generation

In [ ]:
sns.countplot(x=train["Generation"],hue=train["Winner"]);

#### La feature Generation sembrerebbe superflua.  POICHE' I VALORI CHE ASSUMONO WINNER=0 E WINNER=1 SONO PARAGONABILI

### - SPEED

In [ ]:
plt.figure(figsize=(30,30))
sns.countplot(x=train["Speed"],hue=train["Winner"]);

#### Dal grafico di sopra possiamo notare come la distribuzione non sia del tutto omogenea, quindi per capire meglio la relazione:
## - Definisco quindi una nuova Feature che mi rappresenta la differenza fra la stats 'Speed' dei due Pokemon

### - DIFF_SPEED

In [ ]:
train['diff_Speed']=train['Speed']-train['Speed_second']
test['diff_Speed']=test['Speed']-test['Speed_second']

In [ ]:
plt.figure(figsize=(30,30))
sns.countplot(x=train["diff_Speed"],hue=train["Winner"]);

### DA QUESTO GRAFICO RISULTA MOLTO PIU' EVIDENTE COME CAMBIA WINNER IN BASE ALLA VELOCITA'
OVVERO I POKEMON CHE POSSIEDONO VELOCITA' PIU ALTE TENDONO A VINCERE. 
- INFATTI DIFFERENZA POSITIVA VUOL DIRE CHE IL PRIMO E' PIU' VELOCE DEL SECONDO -> VINCE IL PRIMO, E INFATTI WINNER=1
- INVECE DIFFERENZA NEGATIVA VUOL DIRE CHE IL SECONDO E' PIU' VELOCE DEL PRIMO -> VINCE IL SECONDO, E INFATTI WINNER=0

### -HP

In [ ]:
plt.figure(figsize=(50,50))
sns.countplot(x=train["HP"],hue=train["Winner"]);

DA QUESTO GRAFICO SEMBREREBBE CHE HP NON SIA COSI' TANTO UTILE IN MOLTI PUNTI I WINNER=0 O I WINNER=1 SONO PARAGONABILI.

PROVO QUINDI A DEFINIRE UNA NUOVA FEATURE CHE MI RAPPRESENTA LA DIFFERENZA FRA HP DEL POKEMON1 E LA HP DEL POKEMON 2

### -DIFF_HP

In [ ]:
train['diff_HP']=train['HP']-train['HP_second']
test['diff_HP']= test['HP']- test['HP_second']

In [ ]:
plt.figure(figsize=(50,50))
sns.countplot(x=train["diff_HP"],hue=train["Winner"]);

DA QUESTO GRAFICO A PRIMO IMPATTO SEMBREREBBE CHE:
- se il pokemon2 ha HP maggiore del pokemon1 vince il 2° 
- se il pokemon1 ha HP maggiore del pokemon2 vince il 1°


Però questa volta non basta avere HP maggiore per vincere, in quanto per valori di HP vicini allo 0 troviamo che il winner sia paragonabile

### - ATTACK

In [ ]:
plt.figure(figsize=(50,50))

sns.countplot(x=train["Attack"],hue=train["Winner"]);

QUESTO GRAFICO COSI' NON SEMBRA ESSERE MOLTO RILEVANTE: 


VEDIAMO SEMPLICEMENTE CHE I PICCHI SI HANNO, MA PER ALCUNI VALORI DI ATTACO SONO MOLTO PARAGONABILI.


QUINDI DEFINISCO UNA FEATURE CHE MI RAPPRESENTA LA DIFFERENZA FRA I DUE ATTACCHI PER PROVARE AD OTTENERE UN QUALCOSA DI PIU' INTERESSANTE

### - DIFF_ATTACK

In [ ]:
train['diff_Attack']=train['Attack']-train['Attack_second']
test['diff_Attack']=test['Attack']-test['Attack_second']

In [ ]:
plt.figure(figsize=(50,50))
sns.countplot(x=train["diff_Attack"],hue=train["Winner"]);

DA QUESTO GRAFICO SI EVINCE CHE TENDENZIALMENTE:

- ATTACCO 1 MAGGIORE DELL' ATTACCO2--> vince 1
- ATTACCO 2 MAGGIORE DELL' ATTACCO1--> vince2
- PER DIFFERENZE MOLTO PICCOLE NON SI RIESCE A STABILIRE CON CERTEZZA INFATTI COME POSSIAMO VEDERE IN PROSSIMITA' DELLO 0 SONO MOLTO ALTI SIA WINNER=1 SIA WINNER=0


### DEFENSE

In [ ]:
plt.figure(figsize=(50,50))
sns.countplot(x=train["Defense"],hue=train["Winner"]);

I VALORI IN QUESTA DISTRIBUZIONE TRANNE PER ALCUNI PUNTI ISOLATI NON DANNO INFORMAZIONI SIGNIFICATIVE  SONO INFATTI SPESSO MOLTO VICINI I VALORI DI WINNER=0 E I VALORI DI WINNER=1 PER OGNI VALORE DI DIFESA

VEDO SE CON LA DIFFERENZA FRA LE DUE DIFESE QUESTA COSA VIENE CONFERMATA

### -DIFF_DEFENSE

In [ ]:
train['diff_Defense']=train['Defense']-train['Defense_second']
test['diff_Defense']=test['Defense']-test['Defense_second']

In [ ]:
plt.figure(figsize=(50,50))
sns.countplot(x=train["diff_Defense"],hue=train["Winner"]);

DA QUESTO GRAFICO NON SI PUO' DIRE NULLA  I VALORI DI VINCITA DEL PRIMO E DEL SECONDO SONO STATISTICAMENTE PARAGONABILI QUINDI:
#### PROBABILMENTE QUESTA FEATURE PUO' ESSERE DROPPATA

### -SP_ATK

In [ ]:
plt.figure(figsize=(200,50))
sns.countplot(x=train["Sp_Atk"],hue=train["Winner"]);

ANCHE IN QUESTO CASO LA DISTRIBUZIONE DI WINNER TRANNE IN ALCUNI(POCHI) PUNTI SEMBREREBBE PARAGONABILE. 
VEDO SE OTTENGO UNA CONFERMA DEFINENDO UNA NUOVA FEATURE DIFF_SP_ATK

### -DIFF_SP_ATK

In [ ]:
train['diff_Sp_Atk']=train['Sp_Atk']-train['Sp_Atk_second']
test['diff_Sp_Atk']=test['Sp_Atk']-test['Sp_Atk_second']

In [ ]:
plt.figure(figsize=(200,50))
sns.countplot(x=train["diff_Sp_Atk"],hue=train["Winner"]);

DA QUESTO GRAFICO NOTIAMO QUALCOSA SULLE DIFFERENZE DI SPECIAL ATTACK:
- MOLTO NEGATIVE (SPECIAL ATTACK POKEMON 2 MOLTO MAGGIORE RISPETTO ALLO SPECIAL ATTACK DEL POKEMON 1) VINCE IL SECONDO
- MOLTO POSITIVE (SPECIAL ATTACK POKEMON 1 MOLTO MAGGIORE RISPETTO ALLO SPECIAL ATTACK DEL POKEMON 2) VINCE IL PRIMO

SE LO SPECIAL ATTACK 1 E' DI POCO MAGGIORE ALLO SPECIAL ATTACK 2 E VICEVERSA NON SI POSSONO FARE ASSUNZIONI

NON CI SONO TANTISSIMI POKEMON CHE PRESENTANO UNO SPECIAL ATTACK DI GRAN LUNGA MAGGIORE



### -SP_DEF

In [ ]:
plt.figure(figsize=(200,50))
sns.countplot(x=train["Sp_Def"],hue=train["Winner"]);

ANCHE IN QUESTO CASO CI SONO DEI PICCHI NOTIAMO CHE MOLTI POKEMON HANNO COME VALORI DI SPECIAL DEFENSE: 
- 20;25;30;35;40;45;50;55;60;65;90;95;100;105;110;115;120
- SONO POCHI I POKEMON CHE HANNO UNA SPECIAL DEFENSE CON VALORE NON MULTIPLO DI 5
- MA SULL'ANDAMENTO DELLE VITTORE DA QUESTO GRAFICO NON SI CAPISCE MOLTO, PERCUI DEFINISCO UNA NUOVA FEATURE DIFF_SP_DEF

### -DIFF_SP_DEF

In [ ]:
train['diff_Sp_Def']=train['Sp_Def']-train['Sp_Def_second']
test['diff_Sp_Def']=test['Sp_Def']-test['Sp_Def_second']

In [ ]:
plt.figure(figsize=(200,50))
sns.countplot(x=train["diff_Sp_Def"],hue=train["Winner"]);

ANCHE IN QUESTO CASO PER VALORI VICINO ALLO 0 NON SI PUO' DIRE MOLTO, POSSIAMO NOTARE COME PER DIFFERENZE MOLTO GRANDI SIA CHE ESSE SIANO POSITIVE SIA CHE ESSE SIANO NEGATIVE CI SONO POCHISSIMI DATI, E PER BASSE DIFFERENZE QUINDI QUANDO SP_DEF E' PARAGONABILE FRA I DUE POKEMON NON SI PUO' DIRE MOLTO 

### -LEGENDARY

In [ ]:
sns.countplot(x=train["Legendary"],hue=train["Winner"]);

In [ ]:
sns.countplot(x=train["Legendary_second"],hue=train["Winner"]);

Le features Legendary e Legendary_second invece ci mostrano che i legendari sono MOLTO pochi MA tendono a vincere, mentre per i non legendari sembrerebbe che siano molti.

# Mi stampo una heatmap che mi guarda quale sia la correlazione che ho fra le feature a seguito dell'aggiunta delle nuove feature che ho generato, e i drop eseguiti in precedenza


In [ ]:
plt.figure(figsize=(18,10))
correlation = train.corr()
sns.heatmap(correlation, annot = True, linewidth = 2)

Dalla heatmap vediamo che le seguenti feature non sono utili per il nostro scopo:

- Defense (Già confermato in precedenza dal countplot)
- Generation (confermato anche questo dal countplot)
- Defense_second (confermato anche questo dal countplot)
- Generation_second (confermato anche questo dal countplot)
- diff_Defense(Già confermato in precedenza dal countplot)


Quindi si possono droppare


#### Risulta invece che le feature aggiunte riguardanti la differenza fra le varie stats risultano essere abbastanza correlate con la feature Winner







In [ ]:
toDrop = ['Defense','Generation', 'Defense_second', 'Generation_second', 'diff_Defense' ]
train = train.drop(toDrop, axis=1)

In [ ]:
toDrop = ['Defense','Generation', 'Defense_second', 'Generation_second', 'diff_Defense' ]
test = test.drop(toDrop, axis=1)

In [ ]:
plt.figure(figsize=(18,10))
correlation = train.corr()
sns.heatmap(correlation, annot = True, linewidth = 2)

ORA TUTTE LE FEATURE PRESENTANO UNA BUONA CORRELAZIONE CON WINNER

### SEPARO LA LABEL DALLE FEATURE 

In [ ]:
y_train = train['Winner']          
X_train = train.drop(['Winner'], axis=1)      

y_test = test['Winner']           
X_test = test.drop(['Winner'], axis=1)  


### RANDOM FOREST

L’algoritmo come si può dedurre dal nome, è costituito da un insieme di alberi decisionali.
La classificazione delle istanze dei dati si ha muovendosi seguendo gli archi dalla radice (nodo iniziale dell’albero) ad una foglia dell’albero. 
Quest’ultima rappresenta il risultato predetto a partire dai valori dei vari attributi e quindi la classificazione del dato in input.
La caratteristica principale degli alberi decisionali è la loro abilità nello scegliere le features più importanti per poter dividere il set di dati in modo tale che i gruppi risultanti siano più diversi possibili fra di loro e quindi i membri del sottogruppo siano più simili possibili fra loro. 
Ciascuna regola di predizione è rappresentata dal cammino che va dal nodo radice al nodo foglia.
Per quanto riguarda la classificazione ogni singolo albero nella foresta genera una previsione di classe e la classe con maggior numero di voti diventa quindi la previsione del modello. 
Il vantaggio di avere più alberi decisionali risiede nel fatto che un gran numero di alberi non correlati protegge l’errore effettuato dai singoli alberi, in quanto mentre alcuni alberi potranno avere un output errato molti altri lo avranno corretto, quindi come gruppo gli alberi si muoveranno verso la corretta direzione.
In sostanza la maggioranza vince

In [ ]:
randomforest = RandomForestClassifier()
randomforest.fit(X_train, y_train)
y_pred = randomforest.predict(X_test)
acc_randomforest = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_randomforest)
plot_confusion_matrix(randomforest, X_test, y_test, values_format='n', display_labels=['0','1'])
print(classification_report(y_test,y_pred))

La matrice di confusione riportata qui sopra molstra che per la maggiorparte dei dati la classificazione risulta avvenire correttamente, inoltre anche i valori di precision e recall sono nel complesso molto buoni. 
In conclusione posso dire che il modello si è prestato molto bene ai fini del problema analizzato

In [ ]:
models = pd.DataFrame({
    'Model': [ 'Random Forest'],
    'Score': [ acc_randomforest ]})
models.sort_values(by='Score', ascending=False)